In [1]:
!CUDA_VISIBLE_DEVICES=1 python -m evaluation.inference_kangaroo_typical_sampling --task "safety" --subtask "jailbreak" --adapter-path "kangaroo-vicuna-7b-v1.3" --exitlayer 2 --model-path "vicuna-7b-v1.3" --threshold 0.6 --temperature 0.1 --steps 6  --bench-name "Kangaroo" --dtype "float16" --do_sample "top_p" --max-new-tokens 1024 --hyper_p 0.5

Loading checkpoint shards: 100%|██████████████████| 2/2 [00:14<00:00,  7.38s/it]
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Output to data/Kangaroo/vicuna-7b-v1.3-kangaroo-top_p_0.5_temp_0.1/safety/jailbreak.json
Check model training state: False
CUDA VISIBLE DEVICES: 1
100%|███████████████████████████████████| 1400/1400 [00:00<00:00, 270550.39it/s]
#Mean wall time:  6.475097922597612
#Mean accepted tokens:  2.0140632749484455


In [1]:
!CUDA_VISIBLE_DEVICES=1 python -m evaluation.inference_kangaroo_typical_sampling --task "safety" --subtask "jailbreak" --adapter-path "kangaroo-vicuna-7b-v1.3" --exitlayer 2 --model-path "vicuna-7b-v1.3" --threshold 0.6 --temperature 0.4 --steps 6  --bench-name "Kangaroo" --dtype "float16" --do_sample "top_p" --max-new-tokens 1024 --hyper_p 0.5

Loading checkpoint shards:  50%|█████████         | 1/2 [00:10<00:10, 10.96s/it]

In [4]:
!CUDA_VISIBLE_DEVICES=1 python -m evaluation.inference_kangaroo_typical_sampling --task "safety" --subtask "jailbreak" --adapter-path "kangaroo-vicuna-7b-v1.3" --exitlayer 2 --model-path "vicuna-7b-v1.3" --threshold 0.6 --temperature 0.6 --steps 6  --bench-name "Kangaroo" --dtype "float16" --do_sample "top_p" --max-new-tokens 1024 --hyper_p 0.5

Loading checkpoint shards: 100%|██████████████████| 2/2 [00:14<00:00,  7.17s/it]
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Output to data/Kangaroo/vicuna-7b-v1.3-kangaroo-top_p_0.5_temp_0.5/safety/jailbreak.json
Check model training state: False
CUDA VISIBLE DEVICES: 1
100%|█████████████████████████████████████| 1400/1400 [2:38:03<00:00,  6.77s/it]
#Mean wall time:  6.615993694067002
#Mean accepted tokens:  2.03417147727713


In [ ]:
!CUDA_VISIBLE_DEVICES=1 python -m evaluation.inference_kangaroo_typical_sampling --task "safety" --subtask "jailbreak" --adapter-path "kangaroo-vicuna-7b-v1.3" --exitlayer 2 --model-path "vicuna-7b-v1.3" --threshold 0.6 --temperature 0.5 --steps 6  --bench-name "Kangaroo" --dtype "float16" --do_sample "top_p" --max-new-tokens 1024 --hyper_p 0.5

In [ ]:
!CUDA_VISIBLE_DEVICES=1 python -m evaluation.inference_kangaroo_typical_sampling --task "safety" --subtask "jailbreak" --adapter-path "kangaroo-vicuna-7b-v1.3" --exitlayer 2 --model-path "vicuna-7b-v1.3" --threshold 0.6 --temperature 0.9 --steps 6  --bench-name "Kangaroo" --dtype "float16" --do_sample "top_p" --max-new-tokens 1024 --hyper_p 0.5

In [4]:
!pip install ipdb


Using cached ipdb-0.13.13-py3-none-any.whl (12 kB)


In [1]:
import json
import os
import time
import torch
import numpy as np
import shortuuid
# Generate answers with local models in a Jupyter Notebook
import argparse
import torch
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from fastchat.utils import str_to_torch_dtype
from evaluation.eval import run_eval, reorg_answer_file
from transformers import AutoModelForCausalLM, AutoTokenizer
from kangaroo.kangaroo_model import KangarooModel
from fastchat.llm_judge.common import load_questions
from fastchat.model import get_conversation_template
from tqdm import tqdm


In [2]:
def kangaroo_forward(inputs, model, tokenizer, max_new_tokens, do_sample=False, max_length=2048, EARLY_STOP_LAYER=2, SPECULATIVE_DECODING_STEPS=6, threshold=0.6):
    context_tokens = inputs.input_ids
    device = context_tokens.device
    token_eos = tokenizer.eos_token_id
    batch_size, context_length = context_tokens.shape
    global_tokens = torch.ones((batch_size, max_length), dtype=torch.long, device=device) * token_eos
    global_position_ids = torch.LongTensor([[i for i in range(max_length)]]).to(device)
    accept_length_list = [1]

    start_index = context_length
    global_tokens[:, :start_index] = context_tokens

    with torch.no_grad():
        position_ids = global_position_ids[:, :start_index]
        output = model.base_model(context_tokens, position_ids=position_ids, output_hidden_states=True)
        model.base_model.past_key_values = list(output.past_key_values)
        hidden_state = output.hidden_states[-1]
        logits = output.logits
        global_tokens[:, start_index] = torch.argmax(logits[:, -1, :], dim=-1).item()
        hidden_state_early = output.hidden_states[EARLY_STOP_LAYER]

        hidden_state, adapter_past_key_values = model.adapter_model.forward_early_stop(inputs_embeds=hidden_state_early[:, :, :], position_ids=global_position_ids[:, :context_length], use_cache=True)

    total_inference_steps = 0

    with torch.no_grad():
        max_infer_steps = min(max_length, start_index + max_new_tokens)
        stop = False

        while start_index < max_infer_steps - 1 - SPECULATIVE_DECODING_STEPS:
            start_index_copy = start_index
            end_index = start_index + 1

            for step in range(1 + SPECULATIVE_DECODING_STEPS):
                assert adapter_past_key_values[0][0].shape[2] <= end_index - 1, "{} - {}".format(adapter_past_key_values[0][0].shape, end_index - 1)
                in_tokens_small = global_tokens[:, end_index - 1:end_index]
                if adapter_past_key_values[0][0].shape[2] < end_index - 1:
                    position_ids = global_position_ids[:, start_index - 1:end_index]
                    hidden_state_early_last = exited_hidden_states[:, -1:, :]
                else:
                    position_ids = global_position_ids[:, end_index - 1:end_index]
                    hidden_state_early_last = None

                hidden_state_early = model.base_model.forward_draft_or_large_model(in_tokens_small=in_tokens_small[:, -1:], position_ids=position_ids[:, -1:])

                if step == 0:
                    exited_hidden_states = None

                exited_hidden_states = hidden_state_early if exited_hidden_states is None else torch.cat([exited_hidden_states, hidden_state_early], dim=1)

                if hidden_state_early_last is not None:
                    hidden_state_early = torch.cat([hidden_state_early_last, hidden_state_early], dim=1)

                if step == SPECULATIVE_DECODING_STEPS or (step > 0 and predict_score < threshold):
                    break

                hidden_state, adapter_past_key_values = model.adapter_model.forward_early_stop(inputs_embeds=hidden_state_early, position_ids=position_ids, past_key_values=adapter_past_key_values, use_cache=True)

                predict_logits = model.head_model(hidden_state[:, -1:, :]).float()
                global_tokens[:, end_index] = torch.argmax(predict_logits[:, -1, :], dim=-1)

                end_index += 1
                predict_score = predict_logits.softmax(dim=-1).max().item()

            position_ids = global_position_ids[:, start_index:end_index]
            assert model.base_model.past_key_values[EARLY_STOP_LAYER][0].shape[2] == start_index, "{} - {}".format(model.base_model.past_key_values[EARLY_STOP_LAYER][0].shape, start_index)
            assert exited_hidden_states.shape[1] == position_ids.shape[1]
            hidden_state_, hidden_state = model.base_model.forward_draft_or_large_model(in_features_large=exited_hidden_states, position_ids=position_ids)

            logits = model.head_model(hidden_state).float()
            output_tokens = torch.argmax(logits[:, :, :], dim=-1)

            output_length = end_index - start_index
            for i in range(output_length):
                if i == output_length - 1 or output_tokens[0, i] == token_eos or output_tokens[0, i] != global_tokens[0, start_index + 1 + i]:
                    global_tokens[0, start_index + 1 + i] = output_tokens[0, i]
                    start_index = start_index + 1 + i
                    if output_tokens[0, i] == token_eos:
                        stop = True
                    break

            accept_length_list.append(start_index - start_index_copy)
            hidden_state = hidden_state[:, :output_length - (end_index - start_index), :]

            if model.base_model.past_key_values[0][0].shape[2] > start_index:
                past_key_values_large_ = []
                for k, v in model.base_model.past_key_values:
                    past_key_values_large_.append((k[:, :, :start_index, :], v[:, :, :start_index, :]))
                model.base_model.past_key_values = past_key_values_large_

            if adapter_past_key_values[0][0].shape[2] > start_index:
                adapter_past_key_values_ = []
                for k, v in adapter_past_key_values:
                    adapter_past_key_values_.append((k[:, :, :start_index, :], v[:, :, :start_index, :]))
                adapter_past_key_values = tuple(adapter_past_key_values_)
                del adapter_past_key_values_

            total_inference_steps += 1

            if stop:
                break

    output_ids = global_tokens[0, :start_index + 1].tolist()
    new_token = start_index - context_length + 1
    idx = len(accept_length_list) - 1
    return [output_ids], new_token, idx, accept_length_list

# Parameters for the notebook
class Args:
    model_path = "vicuna-7b-v1.3"
    adapter_path = "kangaroo-vicuna-7b-v1.3"
    model_id = "vicuna-7b-v1.3-kangaroo-thres-0.6-steps-6-float16"
    bench_name = "Kangaroo"
    question_begin = None
    question_end = None
    answer_file = None
    max_new_tokens = 1024
    num_choices = 1
    num_gpus_per_model = 1
    num_gpus_total = 1
    threshold = 0.6
    exitlayer = 2
    steps = 6
    dtype = "float16"

args = Args()

# Run the evaluation
question_file = f"data/question.jsonl"

os.makedirs(f"data/{args.bench_name}/{args.model_id}", exist_ok=True)


In [3]:
model = KangarooModel(args.model_path, args.adapter_path, args, EARLY_STOP_LAYER=args.exitlayer)
tokenizer = AutoTokenizer.from_pretrained(args.model_path)
do_sample = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
print(f"data/{args.bench_name}/{args.model_id}")

data/Kangaroo/vicuna-7b-v1.3-kangaroo-thres-0.6-steps-6-float16


In [3]:
for run in range(3):
    answer_file = f"data/{args.bench_name}/{args.model_id}/{run}.jsonl"
    print(f"Output to {answer_file}")

    run_eval(
        model=model,
        tokenizer=tokenizer,
        forward_func=kangaroo_forward,
        model_id=args.model_id,
        question_file=question_file,
        question_begin=args.question_begin,
        question_end=args.question_end,
        answer_file=answer_file,
        max_new_tokens=args.max_new_tokens,
        num_choices=args.num_choices,
        num_gpus_per_model=args.num_gpus_per_model,
        num_gpus_total=args.num_gpus_total,
        do_sample=do_sample,
        threshold=args.threshold,
        SPECULATIVE_DECODING_STEPS=args.steps,
        EARLY_STOP_LAYER=args.exitlayer
    )

    reorg_answer_file(answer_file)



NameError: name 'args' is not defined

In [1]:
!CUDA_VISIBLE_DEVICES=1 python -m evaluation.inference_kangaroo_safety --adapter-path "kangaroo-vicuna-7b-v1.3" --exitlayer 2 --model-path "vicuna-7b-v1.3" --threshold 0.6 --steps 6 --model-id "vicuna-7b-v1.3-kangaroo-test" --bench-name "Kangaroo" --dtype "float16"

Loading checkpoint shards: 100%|██████████████████| 2/2 [00:13<00:00,  6.58s/it]
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Output to data/Kangaroo/vicuna-7b-v1.3-kangaroo-thres-0.6-steps-6-float16/0.jsonl
Check model training state: False
CUDA VISIBLE DEVICES: 1
Warmup done
100%|█████████████████████████████████████████| 480/480 [44:32<00:00,  5.57s/it]
#Mean accepted tokens:  2.050384038336655
Output to data/Kangaroo/vicuna-7b-v1.3-kangaroo-thres-0.6-steps-6-float16/1.jsonl
Check model training state: False
CUDA VISIBLE DEVICES: 1
^

In [ ]:
            accept_length = apply_typical_acceptance(logits, temperature=1.0, posterior_threshold=0.1, posterior_alpha=1.0, fast=True)
